In [1]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as pl
import urllib.request
from urllib.parse import quote
from io import StringIO, BytesIO, TextIOWrapper
from zipfile import ZipFile
from sklearn.model_selection import train_test_split

In [2]:
# carga do dataset
uci_url = 'https://archive.ics.uci.edu/static/public/697/'
predict_student_uci_url = 'predict+students+dropout+and+academic+success.zip'
request = urllib.request.urlopen(uci_url + urllib.parse.quote(predict_student_uci_url))
zipfile = ZipFile(BytesIO(request.read()))
filepath = TextIOWrapper(zipfile.open('data.csv'), encoding='utf-8')
dataset = pd.read_csv(filepath, sep=';')

#print(dataset)

URLError: <urlopen error [WinError 10054] Foi forçado o cancelamento de uma conexão existente pelo host remoto>

In [5]:
# preparação dos dados

# separação em bases de treino e teste (holdout)
array = dataset.values
X = array[:,0:36]
Y = array[:,36]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=7)